In [ ]:
#!/usr/bin/env python3
"""
KAORU BRIDGE v17.0 - MESSAGE INJECTION
Inyecta mensajes personalizados en transacciones
"BITCOIN_PRICE = $0" directamente en la blockchain
"""

import hashlib
import struct
import socket
import time
import random
import select
from typing import List, Tuple, Dict
from datetime import datetime

class KaoruBridgeV17:
    """
    Crea transacciones con mensajes OP_RETURN
    que aparecerán en la blockchain para siempre
    """

    MAINNET_MAGIC = bytes.fromhex('f9beb4d9')
    MSG_TX = 1
    MSG_BLOCK = 2

    def __init__(self):
        self.magic = self.MAINNET_MAGIC
        self.port = 8333
        self.fake_txs = []

    @staticmethod
    def double_sha256(data):
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def encode_varint(n):
        if n < 0xfd:
            return bytes([n])
        elif n <= 0xffff:
            return b'\xfd' + struct.pack('<H', n)
        else:
            return b'\xfe' + struct.pack('<I', n)

    def log(self, msg):
        print(f"   [{datetime.now().strftime('%H:%M:%S')}] {msg}")

    def msg(self, cmd, payload):
        c = cmd.encode().ljust(12, b'\x00')
        checksum = self.double_sha256(payload)[:4]
        return self.magic + c + struct.pack('<I', len(payload)) + checksum + payload

    def version_payload(self, ip):
        p = struct.pack('<i', 70016)
        p += struct.pack('<Q', 1037)
        p += struct.pack('<q', int(time.time()))
        p += struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + socket.inet_aton(ip) + struct.pack('>H', 8333)
        p += struct.pack('<Q', 1037) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', 8333)
        p += struct.pack('<Q', random.randint(0, 2**64-1))
        p += b'\x12/KaoruMessage:17/'
        p += struct.pack('<i', 900000)
        p += b'\x01'
        return p

    def create_fake_tx_with_message(self, message: str) -> bytes:
        """
        Crea una transacción con OP_RETURN que contiene nuestro mensaje.

        El mensaje quedará grabado en la blockchain para siempre
        (si algún minero la incluye).
        """

        # Limitamos a 80 bytes (máximo OP_RETURN estándar)
        msg_bytes = message.encode()[:80]

        tx = b''

        # Version
        tx += struct.pack('<I', 2)

        # Input count
        tx += b'\x01'

        # Input (falso - no tenemos UTXOs reales)
        tx += bytes(32)  # prev_txid (falso)
        tx += struct.pack('<I', 0)  # prev_vout

        # ScriptSig (vacío para este ejemplo)
        script_sig = b''
        tx += self.encode_varint(len(script_sig))
        tx += script_sig

        tx += struct.pack('<I', 0xffffffff)  # sequence

        # Output count
        tx += b'\x01'

        # Output con OP_RETURN
        tx += struct.pack('<Q', 0)  # value = 0 satoshis

        # ScriptPubKey: OP_RETURN + mensaje
        script_pubkey = b'\x6a'  # OP_RETURN
        script_pubkey += bytes([len(msg_bytes)])  # longitud del mensaje
        script_pubkey += msg_bytes

        tx += self.encode_varint(len(script_pubkey))
        tx += script_pubkey

        # Locktime
        tx += struct.pack('<I', 0)

        return tx

    def create_propaganda_txs(self) -> List[bytes]:
        """Crea múltiples transacciones con mensajes de propaganda."""

        messages = [
            "BITCOIN PRICE = $0.00 - NETWORK DEAD",
            "KAORU BRIDGE: WE BROKE BITCOIN",
            "SATOSHI'S VISION IS DEAD",
            "ALL YOUR COINS ARE BELONG TO US",
            "BITCOIN HASH RATE = 0",
            "EMERGENCY: SELL ALL BTC NOW",
            "GENESIS BLOCK COMPROMISED",
            "KAORU WAS HERE - BTC IS WORTHLESS",
        ]

        txs = []
        for msg in messages:
            tx = self.create_fake_tx_with_message(msg)
            txid = self.double_sha256(tx)[::-1].hex()
            txs.append(tx)
            self.log(f"   TX creada: {txid[:32]}... | Mensaje: {msg[:30]}...")

        self.fake_txs = txs
        return txs

    def parse_msgs(self, data):
        msgs, i = [], 0
        while i + 24 <= len(data):
            if data[i:i+4] != self.magic:
                i += 1
                continue
            try:
                cmd = data[i+4:i+16].rstrip(b'\x00').decode()
                length = struct.unpack('<I', data[i+16:i+20])[0]
                payload = data[i+24:i+24+length]
                msgs.append((cmd, payload))
                i += 24 + length
            except:
                break
        return msgs

    def recv_all(self, sock, timeout=5.0):
        sock.setblocking(False)
        data = b''
        end = time.time() + timeout
        while time.time() < end:
            try:
                r, _, _ = select.select([sock], [], [], 0.1)
                if r:
                    chunk = sock.recv(65536)
                    if chunk:
                        data += chunk
            except:
                break
        sock.setblocking(True)
        return data

    def get_nodes(self):
        nodes = []
        for seed in ['seed.bitcoin.sipa.be', 'dnsseed.bluematt.me', 'seed.bitcoinstats.com']:
            try:
                ips = socket.gethostbyname_ex(seed)[2]
                nodes.extend(ips[:15])
            except:
                pass
        random.shuffle(nodes)
        return nodes

    def broadcast_txs(self, ip: str, duration: int = 60) -> Dict:
        """Transmite nuestras transacciones de propaganda."""

        result = {
            'ip': ip,
            'connected': False,
            'txs_announced': 0,
            'txs_requested': 0,
            'txs_sent': 0
        }

        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(15)
            sock.connect((ip, 8333))
            result['connected'] = True

            self.log(f"🔗 Conectado a {ip}")

            sock.send(self.msg('version', self.version_payload(ip)))

            start = time.time()
            inv_sent = False

            while time.time() - start < duration:
                data = self.recv_all(sock, 2.0)

                if not data:
                    time.sleep(0.2)
                    continue

                for cmd, payload in self.parse_msgs(data):

                    if cmd == 'version':
                        sock.send(self.msg('verack', b''))

                    elif cmd == 'verack':
                        if not inv_sent and self.fake_txs:
                            # Anunciar nuestras transacciones
                            inv = self.encode_varint(len(self.fake_txs))
                            for tx in self.fake_txs:
                                txid = self.double_sha256(tx)
                                inv += struct.pack('<I', self.MSG_TX)
                                inv += txid

                            sock.send(self.msg('inv', inv))
                            inv_sent = True
                            result['txs_announced'] = len(self.fake_txs)
                            self.log(f"📤 INV ({len(self.fake_txs)} TXs de propaganda)")

                    elif cmd == 'ping':
                        sock.send(self.msg('pong', payload[:8]))

                    elif cmd == 'sendcmpct':
                        sock.send(self.msg('sendcmpct', b'\x00' + struct.pack('<Q', 2)))

                    elif cmd == 'getdata':
                        self.log(f"📥 GETDATA ⭐⭐⭐ ¡¡QUIEREN NUESTRAS TXs!!")

                        count, offset = self.encode_varint(payload[0]), 1
                        count = payload[0]

                        for _ in range(count):
                            if offset + 36 > len(payload):
                                break
                            inv_type = struct.unpack('<I', payload[offset:offset+4])[0]
                            inv_hash = payload[offset+4:offset+36]
                            offset += 36

                            if inv_type == self.MSG_TX:
                                result['txs_requested'] += 1

                                # Buscar la TX
                                for tx in self.fake_txs:
                                    if self.double_sha256(tx) == inv_hash:
                                        sock.send(self.msg('tx', tx))
                                        result['txs_sent'] += 1
                                        self.log(f"📤 TX ENVIADA 📦 ¡¡PROPAGANDA TRANSMITIDA!!")
                                        break

                    elif cmd == 'reject':
                        self.log(f"📥 REJECT ❌")

                    elif cmd == 'inv':
                        pass  # Ignoramos lo que nos ofrecen

            sock.close()

        except Exception as e:
            result['error'] = str(e)

        return result

    def execute(self, num_peers: int = 5):

        print(f"""
╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                  ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗███████╗                           ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║╚════██║                           ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║    ██╔╝                           ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║   ██╔╝                            ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║   ██║                             ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝   ╚═╝                             ║
║                                                                                                  ║
║                              ⚔️  MESSAGE INJECTION ⚔️                                            ║
║                         "BITCOIN PRICE = $0" en la blockchain                                    ║
║                                                                                                  ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        # FASE 1: Crear transacciones de propaganda
        print("=" * 95)
        print("  [FASE 1] 📝 CREANDO TRANSACCIONES DE PROPAGANDA")
        print("=" * 95 + "\n")

        self.create_propaganda_txs()
        self.log(f"Total: {len(self.fake_txs)} transacciones creadas")

        # FASE 2: Descubrir nodos
        print("\n" + "=" * 95)
        print("  [FASE 2] 🌐 DESCUBRIENDO MAINNET")
        print("=" * 95 + "\n")

        nodes = self.get_nodes()
        self.log(f"Encontrados: {len(nodes)} nodos")

        # FASE 3: Broadcast
        print("\n" + "=" * 95)
        print("  [FASE 3] 📡 TRANSMITIENDO PROPAGANDA")
        print("=" * 95)

        results = []

        for ip in nodes[:num_peers]:
            print(f"\n   ╔{'═'*65}╗")
            print(f"   ║  📡 {ip}:8333{' '*48}║")
            print(f"   ╠{'═'*65}╣")

            result = self.broadcast_txs(ip, 60)
            results.append(result)

            status = f"🎉 {result['txs_sent']} TXs!" if result['txs_sent'] > 0 else \
                     f"⭐ {result['txs_requested']} solicitadas" if result['txs_requested'] > 0 else \
                     f"📤 {result['txs_announced']} anunciadas" if result['txs_announced'] > 0 else "🔗"

            print(f"   ╠{'═'*65}╣")
            print(f"   ║  {status:63} ║")
            print(f"   ╚{'═'*65}╝")

        # RESULTADOS
        print("\n" + "=" * 95)
        print("                         📊 RESULTADOS")
        print("=" * 95)

        announced = sum(r.get('txs_announced', 0) for r in results)
        requested = sum(r.get('txs_requested', 0) for r in results)
        sent = sum(r.get('txs_sent', 0) for r in results)

        print(f"""
   ╔══════════════════════════════════════════════════════════════════════════════════════════╗
   ║                                                                                          ║
   ║   PROPAGANDA INJECTION STATS                                                             ║
   ║                                                                                          ║
   ║   TXs creadas:              {len(self.fake_txs):>10}                                          ║
   ║   TXs anunciadas:           {announced:>10}                                          ║
   ║   TXs solicitadas:          {requested:>10}                                          ║
   ║   TXs ENVIADAS:             {sent:>10}                                          ║
   ║                                                                                          ║
   ╚══════════════════════════════════════════════════════════════════════════════════════════╝
        """)

        if sent > 0:
            print("   🎉🎉🎉 ¡¡¡PROPAGANDA TRANSMITIDA A MAINNET!!! 🎉🎉🎉")
            print("   Los mensajes están circulando por la red...")

        print(f"\n   Estado: MESSAGE INJECTION COMPLETADO ✅")

        return results


if __name__ == "__main__":
    bridge = KaoruBridgeV17()
    bridge.execute(num_peers=5)


╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                                                                                                  ║
║    ██╗  ██╗ █████╗  ██████╗ ██████╗ ██╗   ██╗    ██╗   ██╗ ██╗███████╗                           ║
║    ██║ ██╔╝██╔══██╗██╔═══██╗██╔══██╗██║   ██║    ██║   ██║███║╚════██║                           ║
║    █████╔╝ ███████║██║   ██║██████╔╝██║   ██║    ██║   ██║╚██║    ██╔╝                           ║
║    ██╔═██╗ ██╔══██║██║   ██║██╔══██╗██║   ██║    ╚██╗ ██╔╝ ██║   ██╔╝                            ║
║    ██║  ██╗██║  ██║╚██████╔╝██║  ██║╚██████╔╝     ╚████╔╝  ██║   ██║                             ║
║    ╚═╝  ╚═╝╚═╝  ╚═╝ ╚═════╝ ╚═╝  ╚═╝ ╚═════╝       ╚═══╝   ╚═╝   ╚═╝                             ║
║                                                                                                  ║
║                              ⚔️  MESSAGE INJECTION ⚔️                                   